In [1]:
from jenkspy import JenksNaturalBreaks
import pandas as pd
import numpy as np
import time
import os
import pickle
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
import seaborn as sns
import itertools
import sqlite3

import modules.feature_selection as fs
import modules.helper_functions as hf
import modules.filter_rows as fr

### Helper Functions

All helper functions are included in the module ```helper_functions.py```, such that they can also be used in other notebooks as well.

# Data Preparation

## Feature Engineering and Integration

In [2]:
def get_tmsp_information(data):
    out = data.copy()
    out["month"] = out.copy()["tmsp"].dt.strftime('%b')
    out["dayOfMonth"] = out.copy()["tmsp"].dt.strftime('%#d').astype(int)
    out["weekday"] = out.copy().tmsp.dt.day_name()
    out["weekend"] = np.where(out['weekday'].isin(['Saturday', 'Sunday']), 1, 0)
    out["holiday"] = np.where((out['month'] == 'Jan') & (out['dayOfMonth'] == 1), 1, 0)
    
    return out

def get_daytime(data):
    out = data.copy()
    
    out['time'] = out['tmsp'].dt.strftime('%H:%M')
    out['daytime'] = np.where((out['time'] >= '00:00') & (out['time'] < '06:00'), 'night', 
                        np.where((out['time'] >= '06:00') & (out['time'] < '12:00'), 'morning',
                        np.where((out['time'] >= '12:00') & (out['time'] < '18:00'), 'afternoon', 'evening')))
    out["minuteOfDay"] = (out["tmsp"].dt.hour * 60) + (out["tmsp"].dt.minute)
    
    return out

def get_amountgroup(data, train_length = 0.7, on_training = True):
    out = data.copy()
    out = out.sort_values(by = ["tmsp"], ascending = True)
    out_length = len(out)
    parameters = {}
    
    if on_training:
        train_split = int(np.round(out_length*train_length))
        parameters['train_split_iloc'] = train_split
    else:
        train_split = out_length
    
    amount = list(out.iloc[:train_split,:]['amount'])
    
    jnb = JenksNaturalBreaks(5)
    jnb.fit(amount)
    bins = jnb.breaks_
    bins[0] = 0
    bins[len(bins) - 1] = 10000
    parameters["jenks"] = bins
    hf.writePickle('./data/parameters.pkl', parameters)
    
    out = hf.get_amountgroups(out, bins = bins)
    
    print("= Jenks natural breaks are:")
    print(jnb.breaks_)
    
    return out

def getCard_3DSec_PSP_Amountgroup(data):
    y = hf.get_y(groups=["card", "3D_secured", "PSP", "amountgroup_word"], data = data.copy())
    y = y.sort_values(by=['success_rate'], ascending = False)
    y_mod = y.drop('PSP', axis=1)
    y_mod = y_mod.groupby(["card", "3D_secured", "amountgroup_word"]).aggregate({'success_rate': 'max'}).reset_index()
    y_mod = y_mod.sort_values(by=["success_rate"], ascending = False)
    y_mod = y_mod.merge(y, how="left", on = ["card", "3D_secured", "amountgroup_word", "success_rate"])

    return y_mod

In [3]:
def getOverallSR(data, train_split = 0.7):
    out = data.copy()
    train_split = int(np.round(len(out)*train_split))
    out["overallSR"] = out.iloc[:train_split, :].success.mean()
                
    return out

def combinatoric_SR(data, addColumns = ["PSP", "card", "3D_secured", "amountgroup_word"], train_split = 0.7):
    out = data.copy()

    train_split = int(np.round(len(out)*train_split))
    combinations = {}
    colName = ""
    for col in addColumns:
        combinations[col] = list(out[col].unique())
        colName = colName + col + "_"

    colName = colName + "SR"
    print(colName)
    addColumns.append(colName)

    keys, values = zip(*combinations.items())
    permutations_dicts = [dict(zip(keys, v)) for v in itertools.product(*values)]

    joinFrame = pd.DataFrame()

    i = 1
    for permutation in permutations_dicts:
        subset = out.copy().iloc[:train_split, :]
        for key in permutation.keys():
            subset = subset[subset[key] == permutation[key]]
        subset[colName] = subset.success.mean()
        joinFrame = pd.concat([joinFrame, subset[addColumns]])

    out = out.merge(joinFrame.drop_duplicates(), how = 'left', on = list(set(out.columns).intersection(set(joinFrame.columns))))
    
    return out

def combinatoric_event_window_SR(data, 
                                 addColumns = ["PSP", "card", "3D_secured", "amountgroup_word"], 
                                 event_windows = [5, 10, 100, 200],
                                 allowed_missing = 0.05
                                ):
    out = data.copy()

    for event_window in event_windows:
        print("= Event window size: " + str(event_window))
        combinations = {}
        colName = ""
        replaceCol = ""
        for col in addColumns:
            combinations[col] = list(out[col].unique())
            colName = colName + col + "_"
            replaceCol = replaceCol + col + "_"

        colName = colName + "e" + str(event_window) + "_SR"
        replaceCol = replaceCol + "SR"
        print(colName)
        outCols = addColumns.copy()
        outCols.append(colName)

        keys, values = zip(*combinations.items())
        permutations_dicts = [dict(zip(keys, v)) for v in itertools.product(*values)]

        joinFrame = pd.DataFrame()

        for permutation in permutations_dicts:
            subset = out.copy()
            for key in permutation.keys():
                subset = subset[subset[key] == permutation[key]]
            subset[colName] = subset.success.shift().rolling(
                event_window, min_periods=int(np.ceil(event_window/10))
            ).mean()
            joinFrame = pd.concat([joinFrame, subset[outCols]])
        
        missing_ratio = joinFrame.isna().sum().sum()/len(joinFrame)
        if missing_ratio <= allowed_missing:
            out = out.join(joinFrame[colName])
            out[colName] = out[colName].fillna(out[replaceCol])
        else:
            print("--- Number of missing values too large ---")
    
    return out

def combinatoric_time_window_SR(data,
                                addColumns = ["PSP", "card", "3D_secured", "amountgroup_word"], 
                                time_windows = [1, 6, 12, 24, 72],
                                allowed_missing = 0.15
                                ):
    out = data.copy()

    for time_window in time_windows:
        print("= Time window size: " + str(time_window) + "h")
        combinations = {}
        colName = ""
        replaceCol = ""
        for col in addColumns:
            combinations[col] = list(out[col].unique())
            colName = colName + col + "_"
            replaceCol = replaceCol + col + "_"

        colName = colName + "t" + str(time_window) + "h_SR"
        replaceCol = replaceCol + "SR"
        print(colName)
        outCols = addColumns.copy()
        outCols.append(colName)

        keys, values = zip(*combinations.items())
        permutations_dicts = [dict(zip(keys, v)) for v in itertools.product(*values)]

        joinFrame = pd.DataFrame()

        for permutation in permutations_dicts:
            subset = out.copy()
            for key in permutation.keys():
                subset = subset[subset[key] == permutation[key]]
            subset[colName] = subset[["tmsp", "success"]].rolling(
                            str(time_window) + "h", on = "tmsp", min_periods=int(np.min([np.round(time_window/10), 3]))
                        ).apply(hf.getMeanRollingEvent)["success"]
            joinFrame = pd.concat([joinFrame, subset[outCols]])

        missing_ratio = joinFrame.isna().sum().sum()/len(joinFrame)
        if missing_ratio <= allowed_missing:
            out = out.join(joinFrame[colName])
            out[colName] = out[colName].fillna(out[replaceCol])
        else:
            print("--- Number of missing values too large: " + str(missing_ratio) + " ---")
    
    return out

Bygari, et al. (2021) developed a similar routing approach for an India-based payment service provider called Razorpay. Instead of different payment service providers the usecase has several terminals. For the described "Smart Routing Solution" the authors also proposed to calculate the success rates based on different event- and time-windows. This means that the success rates for a transaction are also calculated based on a rolling-window approach.

Furthermore all window-based feature engineering approaches can be computed overall and based on different features like ```PSP```, ```country``` or ```card```, etc.. This will be limited to the column ```PSP``` for this case-study.

In [4]:
def get_raw_data(dataPath = './data/PSP_Jan_Feb_2019.xlsx', pathDb = './data/PSP_Data.sqlite', table = "TB001_DATA_RAW"):
    
    if not hf.checkIfTableDbExists(pathDb, table):
        out = pd.read_excel(dataPath)
        out = out.drop(["Unnamed: 0"], axis = 1)
        out["tmsp"] = pd.to_datetime(out["tmsp"])
        out = out.sort_values(by = ["tmsp"], ascending = True)

        hf.writeDb(out, pathDb = pathDb, table_name = table)
    else:
        out = hf.readSqlTable(pathDb, table = table)
    
    return out

In [5]:
def applyDataCleaningFeatureEng(dataPath = './data/PSP_Jan_Feb_2019.xlsx', 
                                outPath = './data/data_prepared.csv', 
                                train_length = 0.7, 
                                pathDb = './data/PSP_Data.sqlite'
                               ):
    start_pipeline = time.time()
    
    if not hf.checkIfTableDbExists(pathDb, "TB003_DATA_PREPARED"):
        start_time = time.time()
        print('=== Start raw data loading ===')
        out = get_raw_data(dataPath = dataPath, pathDb = pathDb)
        print("=== Elapsed Time: " + str(time.time() - start_time) + " seconds ===")
        print("Shape of dataframe: " + str(out.shape))

        if not hf.checkIfTableDbExists(pathDb, "TB002_DATA_CLEANED"):
            print("")
            start_time = time.time()
            print('=== Start filter rows ===')
            out = fr.selectRows(out)
            print("=== Elapsed Time: " + str(time.time() - start_time) + " seconds ===")
            print("Shape of dataframe: " + str(out.shape))

            print("")
            start_time = time.time()
            print('=== Start get timestamp information ===')
            out = get_tmsp_information(out)
            out = get_daytime(out)
            print("=== Elapsed Time: " + str(time.time() - start_time) + " seconds ===")
            print("Shape of dataframe: " + str(out.shape))

            print("")
            start_time = time.time()
            print('=== Get amountgroups and daytime by Jenks natural breaks ===')
            out = get_amountgroup(out)
            out = hf.get_daytime(out)
            hf.writeDb(out, pathDb = pathDb, table_name = "TB002_DATA_CLEANED")
            print("=== Elapsed Time: " + str(time.time() - start_time) + " seconds ===")
            print("Shape of dataframe: " + str(out.shape))
        else:
            print("=== Cleaned Data Table already exists - reading from DB ===")
            out = hf.readSqlTable(pathDb, "TB002_DATA_CLEANED")
            print("Shape of dataframe: " + str(out.shape))
        
        print("")
        start_time = time.time()
        print("=== Start Feature Engineering ===")
        print("=== Get overall success rates ===")
        out = getOverallSR(out)
        print("=== Get overall success rates for columns and column combinations")
        print("= PSP")
        out = combinatoric_SR(out, addColumns = ["PSP"])
        print("= PSP x card")
        out = combinatoric_SR(out, addColumns = ["PSP", "card"])
        print("= PSP x card x 3D_secured")
        out = combinatoric_SR(out, addColumns = ["PSP", "card", "3D_secured"])
        print("= PSP x card x 3D_secured x amountgroup_word")
        out = combinatoric_SR(out, addColumns = ["PSP", "card", "3D_secured", "amountgroup_word"])
        print("=== Elapsed Time: " + str(time.time() - start_time) + " seconds ===")
        print("Shape of dataframe: " + str(out.shape))
        
        print("")
        start_time = time.time()
        print("=== Get event window success rates for columns and column combinations")
        print("= PSP")
        out = combinatoric_event_window_SR(out, addColumns = ["PSP"])
        print("= PSP x card")
        out = combinatoric_event_window_SR(out, addColumns = ["PSP", "card"])
        print("= PSP x card x 3D_secured")
        out = combinatoric_event_window_SR(out, addColumns = ["PSP", "card", "3D_secured"])
        print("= PSP x card x 3D_secured x amountgroup_word")
        out = combinatoric_event_window_SR(out, addColumns = ["PSP", "card", "3D_secured", "amountgroup_word"])
        print("=== Elapsed Time: " + str(time.time() - start_time) + " seconds ===")
        print("Shape of dataframe: " + str(out.shape))
        
        print("")
        start_time = time.time()
        print("=== Get time window success rates for columns and column combinations")
        print("= PSP")
        out = combinatoric_time_window_SR(out, addColumns = ["PSP"])
        print("= PSP x card")
        out = combinatoric_time_window_SR(out, addColumns = ["PSP", "card"])
        print("= PSP x card x 3D_secured")
        out = combinatoric_time_window_SR(out, addColumns = ["PSP", "card", "3D_secured"])
        print("= PSP x card x 3D_secured x amountgroup_word")
        out = combinatoric_time_window_SR(out, addColumns = ["PSP", "card", "3D_secured", "amountgroup_word"])
        print("=== Elapsed Time: " + str(time.time() - start_time) + " seconds ===")
        print("Shape of dataframe: " + str(out.shape))
        
        hf.writeDb(out, pathDb = pathDb, table_name = "TB003_DATA_PREPARED")
    
    else:
        print("=== Prepared Data already exists - reading from DB ===")
        out = hf.readSqlTable(pathDb, "TB003_DATA_PREPARED")
     
    print("")
    print("============================")
    print("= time for whole pipeline: " + str(time.time() - start_pipeline) + " seconds")
    print("============================")
    
    return out

In [6]:
data_clean = applyDataCleaningFeatureEng()

=== Table does not exists ===
=== Start raw data loading ===
=== Table does not exists ===
=== Table TB001_DATA_RAW created successful ===
=== Elapsed Time: 6.2141265869140625 seconds ===
Shape of dataframe: (50410, 7)
=== Table does not exists ===

=== Start filter rows ===
= Half time: 152.50623726844788 seconds
=== After cleaning up filtered data: 152 rows removed ===
= End Time: 267.5032305717468 seconds
=== Elapsed Time: 267.5042357444763 seconds ===
Shape of dataframe: (27339, 12)

=== Start get timestamp information ===
=== Elapsed Time: 0.3334932327270508 seconds ===
Shape of dataframe: (27339, 20)

=== Get amountgroups and daytime by Jenks natural breaks ===
= Jenks natural breaks are:
[0, 99, 175, 247, 330, 10000]
=== Table TB002_DATA_CLEANED created successful ===
=== Elapsed Time: 1.1636948585510254 seconds ===
Shape of dataframe: (27339, 21)

=== Start Feature Engineering ===
=== Get overall success rates ===
=== Get overall success rates for columns and column combination

In [7]:
data_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 27339 entries, 0 to 27338
Data columns (total 57 columns):
 #   Column                                        Non-Null Count  Dtype         
---  ------                                        --------------  -----         
 0   tmsp                                          27339 non-null  datetime64[ns]
 1   country                                       27339 non-null  object        
 2   amount                                        27339 non-null  int64         
 3   success                                       27339 non-null  int64         
 4   PSP                                           27339 non-null  object        
 5   3D_secured                                    27339 non-null  int64         
 6   card                                          27339 non-null  object        
 7   failPrevious                                  27339 non-null  float64       
 8   failed_Goldcard                               27339 non-null  floa

The column ```tmsp``` is included in the additionally created columns ```month```, ```dayOfMonth```, ```weekday```, ```holiday```, ```daytime``` and ```minuteOfDay```, so this column can be deleted. Furthermore the column cannot be used in any machine learning model.

The columns ```daytime``` and ```time``` were created for data exploration reasons only. The columns can be completely reproduced by the column ```minuteOfDay```. So also the columns ```daytime``` and ```time``` can also be deleted.

Also the columns ```amountgroup``` and ```amountgroup_word``` can be completely derived from the column ```amount``` and are artifacts from the previous data understanding steps. Highly correlated features containing redundant information can cause problems in many ML settings, so both columns will be removed.

Also the feature ```failPrevious``` which is a dummy-variable to indicate if a transaction has failed previously or not.

The columns ```lower```, ```upper```, ```numLower``` and ```numUpper``` were created for row-selection reasons and can also be excluded as features for the modeling phase.

In [8]:
try:
    print(data_clean[data_clean['PSP'] != "Simplecard"].success.mean())
    data_clean_dropped_woTime = hf.dropColumns(data = data_clean.copy(), 
        columns = ['tmsp_hour', 'daytime', 'time', 'failedPSP', 'amountgroup_word', 'daytime', 'lower', 'upper', 'numUpper'])
except:
    print("=== Object does not exist ===")

0.4191844817955619


## Formatting and Splitting Data

Treebased models are particularly useful in dealing with categorical features and finding insightful breakpoints in continuous variables. From a modeling perspective it seems reasonable to achieve good modeling results with treebased models. Furthermore the dataset is not very large and contains only structured data. In a first step, the data has to be formatted in such a way, that various model implementation in Python can deal with the dataset. Most of the model implementations cannot deal with categorical features. This means all categorical and ordinal features in the dataset have to be enconded to numerical features. This can be achieved by One-Hot or Label-Encoding.

One-Hot encoding is useful, when the categorical variables do not have too many unique values. Label-Encoding can be useful, when the categorical variable have many unique values. Label-Encoding is also used for ordinal variables. So the cardinality of the categorical variables in the dataset has to be inspected first. The categorical features in the dataset are:
* country
* success
* PSP
* 3D_secured
* card
* month
* dayOfMonth
* weekday
* weekend
* holiday

From those variables the following variables are already in a numeric format and can be used in ML models:
* success
* 3D_secured
* dayOfMonth
* weekend
* holiday

The cardinality of the remaining variables are:
* PSP: 4
* card: 3
* month: 2
* weekday: 7

This shows that the yet to be formatted variables have a low cardinality and will be One-Hot-transformed to become useful in terms of modeling purpose.

In [9]:
def formatData(data, columns = ['PSP', 'card', 'month', 'weekday', 'country']):
    out = data.copy()
    
    out = pd.get_dummies(out, columns=columns, drop_first=True)
    print("=== Number of missing values ===")
    print(out.isna().sum().sum())
    
    return out

In [10]:
data_formatted_time = formatData(data_clean_dropped_woTime)

=== Number of missing values ===
0


In [11]:
def getColumnsToScale(data):
    from pandas.api.types import is_numeric_dtype
    out = []
    for column in data.copy().columns:
        if is_numeric_dtype(data.copy()[column]):
            if data.copy()[column].max() > 1:
                out.append(column)
        else:
            print("Column " + column + " is not numeric")
    
    return out

In [12]:
def applyRandomSplitting(data, train_size = 0.7, test_size = 0.15, validate_size = 0.15):
    applyData = data.copy()
    y = applyData['success']
    X = hf.dropColumns(applyData, columns = ["success"])
    
    scale_columns = getColumnsToScale(applyData)
    X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=test_size + validate_size, random_state=1977)
    X_test, X_validate, y_test, y_validate = train_test_split(X_test, y_test, stratify=y_test, 
                                                              test_size=test_size/(test_size + validate_size), 
                                                              random_state=1977)
    scaler = MinMaxScaler()
    X_train[scale_columns] = scaler.fit_transform(X_train[scale_columns])
    X_test[scale_columns] = scaler.transform(X_test[scale_columns])
    X_validate[scale_columns] = scaler.transform(X_validate[scale_columns])
    
    print("= Success rate in y_train: " + str(y_train.sum()/len(y_train)))
    print("= Success rate in y_validate: " + str(y_validate.mean()))
    print("= Success rate in y_test: " + str(y_test.mean()))
    
    return (X, y, X_train, y_train, X_validate, y_validate, X_test, y_test)

In [13]:
def applyTimeSplitting(data, train_size = 0.7, test_size = 0.15, validate_size = 0.15, time_col = "tmsp"):
    applyData = data.copy()
    # applyData = applyData.sort_values(by = [time_col], ascending = True)
    length = len(applyData)
    
    train_length = int(np.round(length*train_size))
    test_length = int(length - train_length)
    validate_length = int(np.round(test_length * (validate_size/(validate_size + test_size))))
    test_length = int(test_length - validate_length)
    
    y = applyData['success']
    X = hf.dropColumns(applyData, columns = ["success", time_col])
    
    assert (test_length + validate_length + train_length) == length, f"number expected: {length}, got: {test_length + validate_length + train_length}"
    
    X_train = X.copy().iloc[:train_length, :]
    y_train = y.copy().iloc[:train_length]
    X_validate = X.copy().iloc[train_length:(train_length + validate_length), :]
    y_validate = y.copy().iloc[train_length:(train_length + validate_length)]
    X_test = X.copy().iloc[(train_length + validate_length):, :]
    y_test = y.copy().iloc[(train_length + validate_length):]
    
    assert (len(X_train) + len(X_validate) + len(X_test)) == length, f"number expected: {length}, got: {(len(X_train) + len(X_validate) + len(X_test))}"
    
    scale_columns = getColumnsToScale(X)
    scaler = MinMaxScaler()
    X_train[scale_columns] = scaler.fit_transform(X_train[scale_columns])
    parameters = hf.loadPickle('./data/parameters.pkl')
    parameters["scaler"] = scaler
    parameters["scale_columns"] = scale_columns
    hf.writePickle('./data/parameters.pkl', parameters)
    X_test[scale_columns] = scaler.transform(X_test[scale_columns])
    X_validate[scale_columns] = scaler.transform(X_validate[scale_columns])
    
    print("= Success rate in y_train: " + str(y_train.mean()))
    print("= Success rate in y_validate: " + str(y_validate.mean()))
    print("= Success rate in y_test: " + str(y_test.mean()))
    
    return (X, y, X_train, y_train, X_validate, y_validate, X_test, y_test)

In [14]:
X, y, X_train, y_train, X_validate, y_validate, X_test, y_test = applyTimeSplitting(data_formatted_time)
if not hf.checkIfTableDbExists('./data/PSP_Data.sqlite', "X"):
    hf.writeDb(X, pathDb = './data/PSP_Data.sqlite', table_name = "X")
    hf.writeDb(y, pathDb = './data/PSP_Data.sqlite', table_name = "y")
if not hf.checkIfTableDbExists('./data/PSP_Data.sqlite', "X_train"):
    hf.writeDb(X_train, pathDb = './data/PSP_Data.sqlite', table_name = "X_train")
    hf.writeDb(y_train, pathDb = './data/PSP_Data.sqlite', table_name = "y_train")
if not hf.checkIfTableDbExists('./data/PSP_Data.sqlite', "X_validate"):
    hf.writeDb(X_validate, pathDb = './data/PSP_Data.sqlite', table_name = "X_validate")
    hf.writeDb(y_validate, pathDb = './data/PSP_Data.sqlite', table_name = "y_validate")
if not hf.checkIfTableDbExists('./data/PSP_Data.sqlite', "X_test"):
    hf.writeDb(X_test, pathDb = './data/PSP_Data.sqlite', table_name = "X_test")
    hf.writeDb(y_test, pathDb = './data/PSP_Data.sqlite', table_name = "y_test")

= Success rate in y_train: 0.38266185922558393
= Success rate in y_validate: 0.35137771275298707
= Success rate in y_test: 0.3550353572299439
=== Table does not exists ===
=== Table X created successful ===
=== Table y created successful ===
=== Table does not exists ===
=== Table X_train created successful ===
=== Table y_train created successful ===
=== Table does not exists ===
=== Table X_validate created successful ===
=== Table y_validate created successful ===
=== Table does not exists ===
=== Table X_test created successful ===
=== Table y_test created successful ===


# References

<p>Bygari, R., Gupta, A., Raghuvanshi, S., Bapna, A., & Sahu, B. (2021). An AI-powered Smart Routing Solution for Payment Systems. 2026–2033. https://doi.org/10.1109/BigData52589.2021.9671961</p>